# 從 API 拿資料

In [34]:
import pandas as pd
import requests
from fake_useragent import UserAgent
import time
from requests.models import PreparedRequest


def fetchData(lastId: str, keyword: str):
    # Dcard 搜尋 API
    baseUrl = 'https://www.dcard.tw/service/api/v2/search/posts'

    # 設定關鍵字與文章數量參數
    params = {'query': keyword, "limit": 50}

    if lastId:
        # 設定延遲時間避免太頻繁的打出 request 造成 service banned
        time.sleep(3)

        # 若資料量不足則設定往前搜尋的參數
        params['before'] = lastId

    req = PreparedRequest()
    req.prepare_url(baseUrl, params)

    ua = UserAgent()

    # 打 API 並取得 response data
    resq = requests.get(req.url, headers={
        "user-agent": ua.random
    })

    return GenerateData(resq.json())


# 產生顯示資料

In [35]:
from datetime import datetime
import calendar


def GenerateData(jsons):
    list = []
    lastId = jsons[len(jsons) - 1]['id']

    for i in range(len(jsons)):
        result = jsons[i]

        # 設定看板名稱
        forumAlias = 'makeup'

        if forumAlias in result['forumAlias']:
            date = datetime.strptime(result['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
            epochTime = calendar.timegm(date.utctimetuple())
            createDate = datetime.fromtimestamp(epochTime).strftime('%Y/%m/%d %H:%M:%S')

            date = datetime.strptime(result['updatedAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
            epochTime = calendar.timegm(date.utctimetuple())
            updateDate = datetime.fromtimestamp(epochTime).strftime('%Y/%m/%d %H:%M:%S')

            id = str(result['id'])
            postUrl = f'https://www.dcard.tw/f/{forumAlias}/p/{id}'

            data = (pd.DataFrame(
                data=[{'ID': id,
                       '標題': result['title'],
                       '摘要':result['excerpt'],
                       '校名匿名':result['anonymousSchool'],
                       '匿名':result['anonymousDepartment'],
                       '發文時間':createDate,
                       '更新時間':updateDate,
                       '回應數量':result['commentCount'],
                       '愛心數量':result['likeCount'],
                       'Tag':result['topics'],
                       'URL':postUrl
                       }],
                columns=['ID', '標題', '摘要', '校名匿名', '匿名', '發文時間', '更新時間', '回應數量', '愛心數量', 'Tag', 'URL']))
            list.append(data)
    return (lastId, list)


# 程式進入點

In [36]:
# 顯示資料
result = []

# 設定關鍵字
keyword = "chifure"

# 最後一組文章 ID
lastId = ""

# 執行 API 與取回顯示資料
while not result:
    (lastId, result) = fetchData(lastId, keyword)

# 組合顯示資料
df = pd.DataFrame()
df = pd.concat(result, ignore_index=True)

# 將顯示資料匯出成 xlsx 檔案
writer = pd.ExcelWriter(f'{keyword}.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name=keyword)
writer.save()

# 於 IDE 顯示資料
df


,ID,標題,摘要,校名匿名,匿名,發文時間,更新時間,回應數量,愛心數量,Tag,URL
0,238670995,#分享 ［保養］韓國平價品牌-Pyunkang Yul-舒敏系列,最近處於過敏階段，因為換季不停的打噴嚏，臉也因此癢泛紅，這一系列都是舒緩系列，適合痘痘肌 敏...,False,False,2022/04/21 00:33:15,2022/04/21 00:33:44,1,38,"[保養, 韓國, 平價, 敏感肌]",https://www.dcard.tw/f/makeup/p/238670995
1,238192854,#分享 遲到但不缺席的2021保養美妝空空賞✨✨,嗨大家我是weiweiii，今天要分享的是2021空空賞～都過奪久了真是🤣，歸納一下去年，整...,False,False,2022/02/23 17:17:27,2022/02/23 17:17:27,7,76,"[保養, 空空賞]",https://www.dcard.tw/f/makeup/p/238192854
2,238094743,#分享 五款臉部乳液使用心得,續上篇的洗顏產品分享，今天要來跟大家分享一下手邊臉部乳液的心得，養好膚質的第二步就是做好面部...,False,True,2022/02/11 17:22:57,2022/02/13 21:34:18,15,346,"[臉部, 乳液, 心得, 保養, 保養品]",https://www.dcard.tw/f/makeup/p/238094743
3,238051108,#分享 愛用眼影/腮紅/唇膏分享（文長）,首次在美妝版發文，請大家多多指教，我是カメ‍️，想分享一下手上的一些彩妝，-，［唇膏］，（橘...,False,False,2022/02/05 20:01:04,2022/02/05 20:02:52,3,65,"[分享, 愛用, 眼影, 美妝]",https://www.dcard.tw/f/makeup/p/238051108
4,237857382,#分享 2021年度空空賞（文～長）,安安～我是木木三，原本想要在2021年末發空空賞，結果懶癌末期發作，就……2022年ㄌ……，...,False,False,2022/01/09 15:34:14,2022/01/09 16:37:40,16,121,"[分享, 空空, 保養]",https://www.dcard.tw/f/makeup/p/237857382
5,237750842,💋唇彩分享💋19隻紅色、深色調大集合 (唇膏篇,安妞～各位好啊‍️我是Rose，身為紅唇代言人 （沒～我不要臉胡謅的，這次就要來介紹最大批的...,False,False,2021/12/25 14:36:51,2021/12/25 14:37:18,12,229,"[唇彩, 分享]",https://www.dcard.tw/f/makeup/p/237750842
6,237735911,#請益 保濕洗臉2選1,最近洗完臉真ㄉ超爆乾（ ´•̥̥̥ω•̥̥̥` ），洗過沒泡泡的舒特膚也洗過有泡泡的SIMP...,False,True,2021/12/23 14:29:15,2021/12/23 14:29:15,8,23,[洗面乳],https://www.dcard.tw/f/makeup/p/237735911
7,237677184,💋唇彩分享💋 18隻自然溫柔色調大集合,安妞～各位好啊‍️我是Rose，繼上次的長篇大論，這次⋯我又來啦，因為真的不知道他們要歸類在...,False,False,2021/12/15 18:26:02,2021/12/20 18:22:17,78,823,"[唇彩, 分享]",https://www.dcard.tw/f/makeup/p/237677184
8,237618202,💋唇彩分享💋15隻橘色調大集合,安妞～各位好啊‍️我是Rose，有看過我之前文章的人應該都知道，本人帶橘的唇彩是手邊偏少的，...,False,False,2021/12/07 19:17:16,2021/12/22 21:14:13,80,823,"[唇彩, 分享]",https://www.dcard.tw/f/makeup/p/237618202
9,237616904,#分享 秋冬開架卸妝愛用分享😋,我的皮膚是極乾肌加上容易乾燥泛紅，很多卸妝產品用完都會有點太乾、不太舒服，尤其冬天卸妝水+化...,False,True,2021/12/07 16:05:38,2021/12/07 16:05:38,11,38,"[分享, 卸妝, Loreal, CHIFURE]",https://www.dcard.tw/f/makeup/p/237616904
